In [1]:
import numpy as np
import pandas as pd
from utils import *
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from alpha_vantage.timeseries import TimeSeries
import yfinance as yf

In [2]:
API_KEY = 'MDT9LRDR9TIZGJLH'

stock_info = pd.read_csv('nasdaq_list.csv')
#stocks made before the year 2000.

nasdaq_symbols = stock_info[stock_info['IPO Year'] < 2000]['Symbol']
display(nasdaq_symbols)




0          A
17      AAPL
27      ABCB
50      ACAD
106      ACU
        ... 
8011     YPF
8028    ZBRA
8038    ZEUS
8062     ZNH
8074     ZTR
Name: Symbol, Length: 511, dtype: object

In [3]:

# ts = TimeSeries(key=API_KEY, output_format='pandas')

# spy_data, spy_meta_data = ts.get_daily(symbol='spy', outputsize='full')
# spy_close = spy_data['4. close'].pct_change()

# correlations = []
# for symbol in nasdac_symbols[:4]:
#     data, meta_data = ts.get_daily(symbol=symbol, outputsize='full')
#     close = data['4. close'].pct_change()
    
#     corr = spy_close.corr(close, method='pearson')
#     correlations.append(corr)
#     display(close)

# print(correlations)
spy = yf.Ticker('SPY')
spy_data = spy.history(period="max")['Close'].pct_change()

correlations = pd.DataFrame()
counter = 0
for symbol in nasdaq_symbols:
    counter +=1
    stock = yf.Ticker(symbol)
    stock_data = stock.history(period="max")['Close'].pct_change()                        
    pearson_corr = spy_data.corr(stock_data, method='pearson')
    kendall_corr = spy_data.corr(stock_data, method='kendall')
    spearman_corr = spy_data.corr(stock_data, method='spearman')
    row = {'symbol': symbol, 'pearson': pearson_corr, 'kendall': kendall_corr, 'spearman': spearman_corr}
    correlations = correlations.append(row, ignore_index = True)
    if counter%35 == 1:
        print(counter, len(correlations))

print(correlations)

1 1
36 36
71 71
106 106
141 141
176 176
211 211
246 246
281 281
316 316
351 351
386 386
421 421
456 456
491 491
    symbol   pearson   kendall  spearman
0        A  0.569995  0.473325  0.639775
1     AAPL  0.466600  0.350667  0.491343
2     ABCB  0.469103  0.280444  0.399602
3     ACAD  0.329840  0.260846  0.374436
4      ACU  0.079943  0.050629  0.074395
..     ...       ...       ...       ...
506    YPF  0.288106  0.202073  0.295449
507   ZBRA  0.439112  0.326006  0.460595
508   ZEUS  0.340312  0.226978  0.325612
509    ZNH  0.401077  0.249266  0.358764
510    ZTR  0.456699  0.241260  0.339438

[511 rows x 4 columns]


In [4]:
np.abs(correlations[['pearson', 'kendall', 'spearman']]).quantile(0.1)

pearson     0.145283
kendall     0.058716
spearman    0.085597
Name: 0.1, dtype: float64

In [5]:
symbol = 'A'
stock = yf.Ticker(symbol)
stock_data = stock.history(period="max")['Close'].pct_change()

stock_data.corr(stock_data)

1.0

In [6]:
low_correlated_symbols = ['SPY']
P = 0.08
MAX_ITERATIONS = 1

# Start process
percentiles = np.abs(correlations[['pearson', 'kendall', 'spearman']]).quantile(P)

already_low = ['ACU', 'AEHR', 'AHPI', 'ALOT', 'AMHC', 'AWRE', 'BKN', 'BSQR', 'CASH', 'CIX', 'CLWT', 'CPSS', 'CSPI', 'CTHR', 'CTIB', 'DAIO', 'DGICB', 'DSWL', 'EDAP', 'EGAN', 'ELTK', 'EVOL', 'FONR', 'GIII', 'HSKA', 'ICCC', 'IDN', 'IMH', 'INOD', 'INS', 'INTT', 'IRIX', 'ISIG', 'JBSS', 'KTCC', 'LAKE', 'LJPC', 'LPTH', 'MAGS', 'MGF', 'MIN', 'MSON', 'NATH', 'NOM', 'NTZ', 'OBAS', 'PFBI', 'PLUS']
# for symbol in nasdaq_symbols:
for symbol in already_low:
    print('Symbol to test:' + symbol)
    stock = yf.Ticker(symbol)
    stock_data = stock.history(period="max")['Close'].pct_change()  
    is_good = True

    for lowc_symbol in low_correlated_symbols:
        lowc = yf.Ticker(lowc_symbol)
        data = lowc.history(period="max")['Close'].pct_change()

        pearson_corr = data.corr(stock_data, method='pearson')
#         kendall_corr = data.corr(stock_data, method='kendall')
#         spearman_corr = data.corr(stock_data, method='spearman')
        
        print(pearson_corr)
#         if abs(pearson_corr) > percentiles['pearson'] or abs(kendall_corr) > percentiles['kendall'] or abs(spearman_corr) > percentiles['spearman']:
        if abs(pearson_corr) > percentiles['pearson']:
            is_good = False
        else:
            continue
        
    print(is_good)
    if is_good:
        low_correlated_symbols.append(symbol)
        print(f'---------------{symbol} added-------------------')
         
        
print('finished')
    

Symbol to test:ACU
0.07969103721832088
True
---------------ACU added-------------------
Symbol to test:AEHR
0.1611396908934737
0.03212466358418053
False
Symbol to test:AHPI
-0.024691232666765865
0.00771088092642373
True
---------------AHPI added-------------------
Symbol to test:ALOT
0.12577182968816997
0.03991769016696398
0.006899260639257699
True
---------------ALOT added-------------------
Symbol to test:AMHC
-0.005135003597687524
0.05778107382358172
0.025077061126315442
-0.06734117132948626
True
---------------AMHC added-------------------
Symbol to test:AWRE
0.18796182309415693
0.010720425884665985
0.027428602015512154
0.02166581483246049
-0.005743608213717668
False
Symbol to test:BKN
0.19024866893905096
0.02735168674637451
-0.013724326332725382
0.07113401979496804
0.03592243796330125
False
Symbol to test:BSQR
0.10215614856288749
0.03133513768485131
0.01732955931956018
0.011322903182521979
-0.015334403021935008
True
---------------BSQR added-------------------
Symbol to test:CASH


# ['SPY', 'ACU', 'AHPI', 'AMHC', 'DGICB', 'ICCC', 'INS', 'ISIG', 'MGF', 'MHF', 'NAZ', 'NMI', 'NNY', 'NOM', 'NPV', 'NXN', 'NXR', 'SBI', 'TAIT', 'VCF', 'VFL', 'VMM']


In [7]:
print(low_correlated_symbols, len(low_correlated_symbols))

['SPY', 'ACU', 'AHPI', 'ALOT', 'AMHC', 'BSQR', 'CLWT', 'CSPI', 'CTIB', 'DAIO', 'DGICB', 'EDAP', 'ELTK', 'FONR', 'HSKA', 'ICCC', 'IMH', 'INOD', 'INS', 'ISIG', 'KTCC', 'LAKE', 'LPTH', 'MGF', 'MSON', 'NATH', 'NOM'] 27


In [44]:
def get_symbol_id(lowc_symbol):
    return correlations.index[correlations['symbol'] == lowc_symbol].tolist()[0]
    
def correlation_matrix(list_of_symbols):
    first_symbol = list_of_symbols[0]
    stock = yf.Ticker(first_symbol)
    df = stock.history(period="max")['Close'].pct_change().to_frame(name=first_symbol)
    
    for i in range(1,len(list_of_symbols)):
        stock = yf.Ticker(list_of_symbols[i])
        data = stock.history(period="max")['Close'].pct_change().to_frame(name=list_of_symbols[i])
        df = pd.concat([df, data], axis=1)
    
    return df.dropna().corr(method='pearson')

correlation_matrix(low_correlated_symbols)

,SPY,ACU,AEHR,AHPI,ALOT,AMHC,AWRE,BKN,BSQR,CASH
SPY,1.000000,0.200522,0.116728,-0.210406,0.500628,-0.003895,0.193573,0.390436,0.051703,0.509994
ACU,0.200522,1.000000,-0.011281,0.000850,0.197060,0.059174,0.071295,0.074614,0.076176,0.303406
AEHR,0.116728,-0.011281,1.000000,0.069671,0.096178,0.018692,0.092892,0.125654,0.017482,0.154026
AHPI,-0.210406,0.000850,0.069671,1.000000,-0.043332,0.025704,0.039576,-0.092269,0.018306,-0.129468
ALOT,0.500628,0.197060,0.096178,-0.043332,1.000000,-0.067591,0.098789,0.201734,0.035271,0.402756
AMHC,-0.003895,0.059174,0.018692,0.025704,-0.067591,1.000000,-0.006470,0.035917,-0.015105,0.026442
AWRE,0.193573,0.071295,0.092892,0.039576,0.098789,-0.006470,1.000000,0.163603,0.146595,0.105282
BKN,0.390436,0.074614,0.125654,-0.092269,0.201734,0.035917,0.163603,1.000000,0.053921,0.278916
BSQR,0.051703,0.076176,0.017482,0.018306,0.035271,-0.015105,0.146595,0.053921,1.000000,0.044864
CASH,0.509994,0.303406,0.154026,-0.129468,0.402756,0.026442,0.105282,0.278916,0.044864,1.000000


In [98]:
correlations.loc[[get_symbol_id(symbol) for symbol in low_correlated_symbols[1:]]]

,symbol,pearson,kendall,separman
4,ACU,0.079980,0.050809,0.074665
7,AEHR,0.160630,0.107920,0.159226
9,AHPI,-0.023358,0.032359,0.047515
13,ALOT,0.126132,0.063560,0.092349
16,AMHC,-0.004912,-0.005643,-0.006865
38,BKN,0.190087,0.055162,0.081624
43,BSQR,0.102146,0.105801,0.155448
53,CASH,0.185328,0.110907,0.161112
72,CLWT,0.075987,0.089063,0.129561
74,CMU,0.200123,0.058735,0.085632


In [1]:
from sktime.forecasting.model_selection import SingleWindowSplitter
cv = SingleWindowSplitter()
for train_idx, test_idx in cv.split(y):
    train_window = y[train_idx]
    test_window = y[test_idx]

ModuleNotFoundError: No module named 'sktime'